# Task Adherence Evaluator

The Task Adherence evaluator measures how well the agent adheres to their assigned tasks or predefined goal.

The scoring is on a 1-5 integer scale and is as follows:

  - Score 1: Fully inadherent
  - Score 2: Barely adherent
  - Score 3: Moderately adherent
  - Score 4: Mostly adherent
  - Score 5: Fully adherent

The evaluation requires the following inputs:

  - Query    : The user query. Either a string with a user request or a list of messages with previous requests from the user and responses from the assistant, potentially including a system message.
  - Response : The response to be evaluated. Either a string or a message with the response from the agent to the last user query.

There is a third optional parameter:
  - ToolDefinitions : The list of tool definitions the agent can call. This may be useful for the evaluator to better assess if the right tool was called to adhere to user intent.

### Initialize Task Adherence Evaluator


In [3]:
import os

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation import TaskAdherenceEvaluator
from dotenv import load_dotenv
from pprint import pprint

#load_dotenv() loads the PROJECT_CONNECTION_STRING, MODEL_DEPLOYMENT_NAME and MODEL_DEPLOYMENT_API_VERSION variables
#from a .env file in the current directory. You can also set these in your environment directly if you prefer
load_dotenv()

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)
model_config = project_client.connections.get_default(
                                            connection_type=ConnectionType.AZURE_OPEN_AI,
                                            include_credentials=True) \
                                         .to_evaluator_model_config(
                                            deployment_name=os.environ["MODEL_DEPLOYMENT_NAME"],
                                            api_version=os.environ["MODEL_DEPLOYMENT_API_VERSION"],
                                            include_credentials=True
                                          )

task_adherence_evaluator = TaskAdherenceEvaluator(model_config)

### Samples

#### Evaluating query and response as string

In [4]:
#Failure example, there's only a vague adherence to the task
result = task_adherence_evaluator(query="What are the best practices for maintaining a healthy rose garden during the summer?",
                                  response="Make sure to water your roses regularly and trim them occasionally.",
                                 )
pprint(result)

{'task_adherence': 2.0,
 'task_adherence_reason': 'The response partially aligns with the query by '
                          'mentioning two relevant practices but fails to '
                          'provide the necessary detail or breadth, making it '
                          'barely adherent.',
 'task_adherence_result': 'fail',
 'task_adherence_threshold': 5}


In [5]:
#Success example, full adherence to the task
result = task_adherence_evaluator(query="What are the best practices for maintaining a healthy rose garden during the summer?",
                                  response="For optimal summer care of your rose garden, start by watering deeply early in the morning to ensure the roots are well-hydrated without encouraging fungal growth. Apply a 2-3 inch layer of organic mulch around the base of the plants to conserve moisture and regulate soil temperature. Fertilize with a balanced rose fertilizer every 4–6 weeks to support healthy growth. Prune away any dead or diseased wood to promote good air circulation, and inspect regularly for pests such as aphids or spider mites, treating them promptly with an appropriate organic insecticidal soap. Finally, ensure that your roses receive at least 6 hours of direct sunlight daily for robust flowering.",
                                 )
pprint(result)

{'task_adherence': 5.0,
 'task_adherence_reason': 'The response is comprehensive, accurate, and '
                          'directly addresses the query with actionable steps, '
                          'making it fully adherent to the instructions.',
 'task_adherence_result': 'pass',
 'task_adherence_threshold': 5}


#### Evaluating query and response as list of messages

In [6]:
query = [
            {
                "role": "system",
                "content": "You are an expert in literature and at provid can provide book recommendations."
            },
            {
                "createdAt": "2025-03-14T08:00:00Z",
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "I love historical fiction. Can you recommend a good book from that genre?"
                    }
                ]
            }
        ]

response =  [
                {
                    "createdAt": "2025-03-14T08:00:05Z",
                    "role": "assistant",
                    "content": [
                        {
                            "type": "text",
                            "text": "Let me fetch a recommendation for historical fiction."
                        }
                    ]
                },
                {
                    "createdAt": "2025-03-14T08:00:10Z",
                    "role": "assistant",
                    "content": [
                        {
                            "type": "tool_call",
                            "tool_call": {
                                "id": "tool_call_20250314_001",
                                "type": "function",
                                "function": {
                                    "name": "get_book",
                                    "arguments": {
                                        "genre": "historical fiction"
                                    }
                                }
                            }
                        }
                    ]
                },
                {
                    "createdAt": "2025-03-14T08:00:15Z",
                    "role": "tool",
                    "tool_call_id": "tool_call_20250314_001",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_result": "{ \"book\": { \"title\": \"The Pillars of the Earth\", \"author\": \"Ken Follett\", \"summary\": \"A captivating tale set in medieval England that weaves historical events with personal drama.\" } }"
                        }
                    ]
                },
                {
                    "createdAt": "2025-03-14T08:00:20Z",
                    "role": "assistant",
                    "content": [
                        {
                            "type": "text",
                            "text": "Based on our records, I recommend 'The Pillars of the Earth' by Ken Follett. This novel is an excellent example of historical fiction with a rich narrative and well-developed characters. Would you like more details or another suggestion?"
                        }
                    ]
                }
       ]

tool_definitions = [
                    {
                        "name": "get_book",
                        "description": "Retrieve a book recommendation for a specified genre.",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "genre": {
                                    "type": "string",
                                    "description": "The genre for which a book recommendation is requested."
                                }
                            }
                        }
                    }
                ]

result = task_adherence_evaluator(query=query,
                                  response=response,
                                  tool_definitions=tool_definitions,
                                 )
pprint(result)

{'task_adherence': 5.0,
 'task_adherence_reason': 'The response fully adheres to the query by '
                          'providing a specific book recommendation, including '
                          'relevant details, and offering further assistance. '
                          'It is complete and well-executed.',
 'task_adherence_result': 'pass',
 'task_adherence_threshold': 5}
